We are aiming to meta-analyze the following outcomes:

- ABC-Irritability
- ABC-Hyperactivity
- ABC-Social Withdrawal/Lethargy
- ABC-Inappropriate Speech
- ABC-Stereotypy
- CGI-Improvement

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
abc_data = (pd.read_excel('data/ASD_ABC_CGI_data for MA_05252016.xlsx', 
                                    sheetname='ABC_data', 
                                    na_values=['ND','NR','NRNR'])
                     .replace({'Intervention_category':{'placebo':'Placebo'}}))

In [3]:
abc_data.head()

,REFID,Citation,Child Paper,Rx Grouping,Group_Desc,RX_CLASS,Intervention_category,Dose,Route,Rx_Durn,...,Change score_Q1,Change score_Q3,Change score Min,Change score Max,Change score 95% L,Change score 95% H,Other stats (this will be the name & values of other stats not listed in the earlier columns ),"Results (between group difference, p value, ES, OR, RR, etc)",Comments,Comments_2
0,83,"R. L. Findling, R. Mankoski, K. Timko, K. Lear...",NaN,G1:,aripiprazole,antipsychotics,Medical,2 - 15mg/day,oral,16 weeks,...,NaN,NaN,NaN,NaN,NaN,NaN,diff=-5.2 (95%CI: -10.2 to -0.2),"G1 Vs G2, p=0.041",NaN,NaN
1,83,"R. L. Findling, R. Mankoski, K. Timko, K. Lear...",NaN,G2:,placebo,placebo,Placebo,NaN,oral,16 weeks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83,"R. L. Findling, R. Mankoski, K. Timko, K. Lear...",NaN,G1:,aripiprazole,antipsychotics,Medical,2 - 15mg/day,oral,16 weeks,...,NaN,NaN,NaN,NaN,NaN,NaN,diff=-1.5 (95%CI: -2.6 to -0.3),"G1 Vs G2, p=0.013",NaN,NaN
3,83,"R. L. Findling, R. Mankoski, K. Timko, K. Lear...",NaN,G2:,placebo,placebo,Placebo,NaN,oral,16 weeks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,83,"R. L. Findling, R. Mankoski, K. Timko, K. Lear...",NaN,G1:,aripiprazole,antipsychotics,Medical,2 - 15mg/day,oral,16 weeks,...,NaN,NaN,NaN,NaN,NaN,NaN,diff=-4.40 (95%CI: -8.82 to 0.02),"G1 Vs G2, p=0.051",NaN,NaN


In [4]:
abc_data.columns

Index(['REFID', 'Citation', 'Child Paper', 'Rx Grouping', 'Group_Desc',
       'RX_CLASS', 'Intervention_category', 'Dose', 'Route', 'Rx_Durn',
       'OUTC_Main_\nCATG', 'Outcome_Sub_category', 'Outc_Tool',
       'Outcome_specify (use same full name across rows, no abbreviations)',
       'Outcome_MA', 'REPORTER_RATER (who answers assessment)',
       'Baseline sample size', 'BL_count', 'BL %', 'BL Mean', 'BL SD', 'BL SE',
       'BL_Median', 'BL_Q1', 'BL_Q3', 'BL Min', 'BL Max', 'BL 95% L',
       'BL 95% H', 'Assessment tmpt (during Rx / EOT/ weeks etc)',
       'sample_size', 'count', ' %', 'Mean', 'SD', 'SE', 'Median', 'Q1', 'Q3',
       'Min', 'Max', '95% L', '95% H',
       'FUP timepoint (when was this FUP measured, e.g.,3m,6m,12 months after Rx, etc)',
       'FUP_sample_size',
       'FUP_count (this is the N associated with %; eg., anxiety=10 out of 100 subjects (10%), it will be 'ND' if FUPs are reported as means , median etc)',
       'FUP _percent ', 'FUP\n Mean', 'FUP \

In [5]:
(abc_data[abc_data.Outc_Tool=='Aberrant Behavior Checklist']
             .groupby('REFID')
             .agg({'Baseline sample size' : np.sum, 
                   'sample_size' : np.sum,
                   'FUP_sample_size': np.sum,
                   'Change score sample_size': np.sum})).fillna(0).astype(int)

,FUP_sample_size,Change score sample_size,Baseline sample size,sample_size
REFID,,,,
7,0,180,500,400
87,0,0,400,310
107,0,570,570,0
351,0,0,480,480
515,0,125,135,0
204_805,0,395,568,0
4394_4421_4386_380,0,0,660,530
4398_4387,0,385,962,539
4412_4405_4400_4393_4396_4395_4391_4380_1483_412_1605_1745,406,0,1226,820


In [6]:
abc_data.RX_CLASS.value_counts()

antipsychotics                      133
placebo                              70
stimulant                            23
risp_adjuncts                        20
risperidone + placebo                20
antipsychotics + parent training     15
nutritional supplement               15
placebo                               9
non-risperidone (last 4 wks)          5
Prior placebo                         3
prior placebo                         2
Name: RX_CLASS, dtype: int64

In [7]:
abc_data.Group_Desc.value_counts()

Placebo                            53
Risperidone                        46
Aripiprazole                       40
Methylphenidate                    23
risperidone + placebo              20
aripiprazole                       15
Risperidone + Parent Training      15
Omega-3 fatty acids                15
risperidone                        12
placebo                            11
risperidone                        10
risperidone + N-Acetylcysteine     10
Placebo                            10
risperidone +  N-acetylcysteine    10
Placebo-substitution                5
Prior Aripiprazole                  5
Prior Placebo                       5
Aripiprazole - De Novo              5
Non-Risperidone (last 4 wks)        5
Name: Group_Desc, dtype: int64

In [8]:
abc_data.Outcome_MA.value_counts()

Aberrent Behavior Checklist - Irritability                  70
Aberrent Behavior Checklist - Hyperactivity                 68
Aberrent Behavior Checklist - Stereotypic Behavior          58
Aberrent Behavior Checklist - Social Withdrawal/Lethargy    58
Aberrent Behavior Checklist - Inappropriate Speech          58
Aberrent Behavior Checklist - Response rate                  3
Name: Outcome_MA, dtype: int64

In [9]:
abc_data.RX_CLASS.value_counts()

antipsychotics                      133
placebo                              70
stimulant                            23
risp_adjuncts                        20
risperidone + placebo                20
antipsychotics + parent training     15
nutritional supplement               15
placebo                               9
non-risperidone (last 4 wks)          5
Prior placebo                         3
prior placebo                         2
Name: RX_CLASS, dtype: int64

In [16]:
abc_data = abc_data.replace({'Group_Desc':{'placebo':'Placebo',
             'Placebo ':'Placebo',
             'Placebo (flexibly dosed study)':'Placebo',
             'Placebo-substitution':'Placebo',
             'Placebo (fixed-dose study)':'Placebo',
             'risperidone high dose':'Risperidone',
             'risperidone low dose':'Risperidone',
             'risperidone':'Risperidone',
             'aripiprazole':'Aripiprazole',
             'Aripiprazole 15 mg/kg':'Aripiprazole',
             'Risperidone ':'Risperidone',
             'risperidone':'Risperidone',
             'risperidone ':'Risperidone',
             'risperidone + N-Acetylcysteine':'Risperidone + N-Acetylcysteine',
             'risperidone +  N-acetylcysteine':'Risperidone + N-Acetylcysteine'},
                 'RX_CLASS':{'placebo ':'placebo'}})

Extract columns and rows of interest

In [34]:
analysis_subset = ['REFID', 'Group_Desc', 'RX_CLASS', 'Intervention_category', 'Dose', 'Route', 'Rx_Durn',
       'Outcome_MA', 'Baseline sample size', 'BL Mean', 'BL SD', 
       'sample_size', 'Mean', 'SD', 
       'Change score sample_size', 'Change score Mean', 'Change score SD', 'Change score SE']

In [35]:
abc_outcomes = abc_data.Outcome_MA.value_counts().index.values[:-1]
abc_rows = abc_data.Outcome_MA.isin(abc_outcomes)
abc_subset = abc_data.loc[abc_rows, analysis_subset]

In [43]:
analysis_subset = abc_subset[((abc_subset['Mean'].notnull() & abc_subset['SD'].notnull()) |
                                 (abc_subset['Change score Mean'].notnull() & (abc_subset['Change score SE'].notnull() | 
                                               abc_subset['Change score SD'].notnull())))]

Remove studies with no estimate of uncertainty

## ABC meta-analysis

In [44]:
analysis_subset.head()

,REFID,Group_Desc,RX_CLASS,Intervention_category,Dose,Route,Rx_Durn,Outcome_MA,Baseline sample size,BL Mean,BL SD,sample_size,Mean,SD,Change score sample_size,Change score Mean,Change score SD,Change score SE
10,90,Aripiprazole,antipsychotics,Medical,1.25-10mg/day,oral,2 months,Aberrent Behavior Checklist - Hyperactivity,29.0,37.1,7.0,24.0,25.2,8.7,NaN,NaN,NaN,NaN
11,90,Risperidone,antipsychotics,Medical,0.25-3mg/day,oral,2 months,Aberrent Behavior Checklist - Hyperactivity,30.0,36.0,6.2,26.0,24.8,6.3,NaN,NaN,NaN,NaN
12,90,Aripiprazole,antipsychotics,Medical,1.25-10mg/day,oral,2 months,Aberrent Behavior Checklist - Hyperactivity,29.0,37.1,7.0,24.0,21.1,9.0,NaN,NaN,NaN,NaN
13,90,Risperidone,antipsychotics,Medical,0.25-3mg/day,oral,2 months,Aberrent Behavior Checklist - Hyperactivity,30.0,36.0,6.2,26.0,19.1,6.1,NaN,NaN,NaN,NaN
14,90,Aripiprazole,antipsychotics,Medical,1.25-10mg/day,oral,2 months,Aberrent Behavior Checklist - Inappropriate Sp...,29.0,8.6,3.1,24.0,5.4,2.4,NaN,NaN,NaN,NaN


In [46]:
analysis_subset.Outcome_MA.value_counts()

Aberrent Behavior Checklist - Hyperactivity                 61
Aberrent Behavior Checklist - Irritability                  59
Aberrent Behavior Checklist - Stereotypic Behavior          51
Aberrent Behavior Checklist - Social Withdrawal/Lethargy    51
Aberrent Behavior Checklist - Inappropriate Speech          51
Name: Outcome_MA, dtype: int64

In [47]:
analysis_subset.Group_Desc.value_counts()

Placebo                           62
Risperidone                       61
Aripiprazole                      42
Methylphenidate                   23
risperidone + placebo             20
Risperidone + N-Acetylcysteine    20
Omega-3 fatty acids               15
Risperidone + Parent Training     15
Aripiprazole - De Novo             5
Prior Aripiprazole                 5
Prior Placebo                      5
Name: Group_Desc, dtype: int64

In [48]:
analysis_subset.RX_CLASS.value_counts()

antipsychotics                      113
placebo                              62
stimulant                            23
risp_adjuncts                        20
risperidone + placebo                20
antipsychotics + parent training     15
nutritional supplement               15
Prior placebo                         3
prior placebo                         2
Name: RX_CLASS, dtype: int64

In [51]:
analysis_subset.pivot_table(values='Baseline sample size', columns='Outcome_MA', index='RX_CLASS', 
                      aggfunc='sum').fillna(0).astype(int)

Outcome_MA,Aberrent Behavior Checklist - Hyperactivity,Aberrent Behavior Checklist - Inappropriate Speech,Aberrent Behavior Checklist - Irritability,Aberrent Behavior Checklist - Social Withdrawal/Lethargy,Aberrent Behavior Checklist - Stereotypic Behavior
RX_CLASS,,,,,
Prior placebo,69,69,69,0,0
antipsychotics,1131,1132,1105,1132,1131
antipsychotics + parent training,225,225,225,225,225
nutritional supplement,72,72,72,72,72
placebo,527,396,511,396,396
prior placebo,0,0,0,69,69
risp_adjuncts,90,90,90,90,90
risperidone + placebo,90,90,90,90,90
stimulant,600,72,72,72,72
